# 3.4 Data Sharing
Science is much more impactful once it's shared. Therefore, we are going to learn how to 
open up our scientific output from a cloud platform, so that is openly available - and 
has the chance to make the impact it should.
- Load data
- ...


Set configurations

In [1]:
import openeo
import numpy as np
import leafmap

### Select Area of Interest
- select a point of interest

In [2]:
import math

# Function to create a bounding box around a point
def create_bounding_box(latitude, longitude, distance_km):
    # Radius of the Earth in kilometers
    earth_radius_km = 6371

    # Convert latitude and longitude from degrees to radians
    lat_rad = math.radians(latitude)
    lon_rad = math.radians(longitude)

    # Calculate the angular distance covered by the given distance_km
    angular_distance = distance_km / earth_radius_km

    # Calculate the latitude and longitude offsets
    lat_offset = math.degrees(angular_distance)
    lon_offset = math.degrees(angular_distance / math.cos(lat_rad))

    # Calculate the coordinates of the southwest and northeast corners
    sw_lat = latitude - lat_offset
    sw_lon = longitude - lon_offset
    ne_lat = latitude + lat_offset
    ne_lon = longitude + lon_offset

    # Return the bounding box as a tuple (sw_lat, sw_lon, ne_lat, ne_lon)
    return (sw_lat, sw_lon, ne_lat, ne_lon)

In [3]:
m = leafmap.Map(center=(46.497012, 11.356429), zoom=14)
m

Map(center=[46.497012, 11.356429], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title',…

In [4]:
import geopandas as gpd
import shapely
from shapely.geometry import Polygon

distance_km = 1 # Distance in kilometers

feat = m.draw_features
geom = feat[0]['geometry']['coordinates']

# Create a bounding box around the point
bbox = create_bounding_box(geom[0], geom[1], distance_km)

# Create polygon from lists of points
x = [bbox[0], bbox[0], bbox[2], bbox[2], bbox[0]]
y = [bbox[1], bbox[3], bbox[3], bbox[1], bbox[1]]

poly = Polygon(zip(x,y))

gs = gpd.GeoSeries.from_wkt([str(poly)])
gdf = gpd.GeoDataFrame({"col1": ["bbox"]}, geometry=gs, crs=4326)
m.add_gdf(gdf)

### Recreate process graph

In [5]:
conn = openeo.connect('https://openeo.dataspace.copernicus.eu/').authenticate_oidc()

collection      = 'SENTINEL2_L2A'
spatial_extent  = {'west':bbox[0],'east':bbox[2],'south':bbox[1],'north':bbox[3],'crs':4326}
temporal_extent = ["2018-02-10", "2018-02-12"]
bands           = ['B03']
s2cube = conn.load_collection(collection,
                          spatial_extent=spatial_extent,
                          bands=bands,
                          temporal_extent=temporal_extent)

Authenticated using refresh token.


In [50]:
s2cube.download("sample.nc")

In [24]:
import xarray as xr
xr.open_dataset("sample.nc",decode_coords="all")

<xarray.Dataset>
Dimensions:  (t: 1, x: 145, y: 209)
Coordinates:
  * t        (t) datetime64[ns] 2018-02-11
  * x        (x) float64 6.787e+05 6.787e+05 6.788e+05 ... 6.802e+05 6.802e+05
  * y        (y) float64 5.153e+06 5.153e+06 5.153e+06 ... 5.151e+06 5.151e+06
    crs      |S1 ...
Data variables:
    B03      (t, y, x) float32 ...
Attributes:
    Conventions:  CF-1.9
    institution:  openEO platform